In [ ]:
import random

import matplotlib.pyplot as plt

import networkx as nx

import xarray as xr

import numpy as np

from deap_er import base, creator, tools

from py_wake.examples.data.iea37 import IEA37Site, IEA37_WindTurbines
from py_wake.literature import Jensen_1983

from evolve import evolve

In [ ]:
site = IEA37Site()

# Modify site with uniform wind probabilities
wd = site.ds["wd"]
p = np.ones_like(wd) / len(wd)
site.ds["P"] = xr.DataArray(p, coords={"wd": wd})

turbines = IEA37_WindTurbines()
model = Jensen_1983(site, turbines)

In [ ]:
if not hasattr(creator, "FitnessMulti"):
    creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

In [ ]:
toolbox = base.Toolbox()

boundary_length = site.boundary_radius / (2 ** 0.5)
toolbox.register("attr_coordinate", random.uniform, -boundary_length, boundary_length)

MIN_TURBINES = 16
MAX_TURBINES = 25
def individual():
    num_attrs = random.randint(MIN_TURBINES, MAX_TURBINES) * 2
    return creator.Individual(
        [toolbox.attr_coordinate() for _ in range(num_attrs)]
    )
toolbox.register("individual", individual)

toolbox.register("population", tools.init_repeat, list, toolbox.individual)

In [ ]:
COST_PER_TURBINE = 2_000_000
COST_PER_METER_WIRING = 50

def euclidean_distance(x1, y1, x2, y2):
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

max_cost = 0
max_output = 0

def evaluate(ind):
    global max_cost, max_output

    x, y = ind[::2], ind[1::2]
    result = model(x, y)
    aep_gwh = result.aep().sum().values

    total_turbine_cost = COST_PER_TURBINE * len(ind) / 2
    
    graph = nx.Graph()

    pairs = list(zip(x, y))
    for a, p in enumerate(pairs):
        graph.add_node(a, pos=p)

    for a in range(len(pairs)):
        for b in range(a + 1, len(pairs)):
            x1, y1 = pairs[a]
            x2, y2 = pairs[b]
            dist = euclidean_distance(x1, y1, x2, y2)
            graph.add_edge(a, b, dist=dist)

    mst = nx.minimum_spanning_tree(graph, weight='dist')
    total_wiring_cost = mst.size(weight="dist") * COST_PER_METER_WIRING

    max_output = max(aep_gwh, max_output)
    max_cost = max(total_turbine_cost + total_wiring_cost, max_cost)

    return aep_gwh * 1_000_000, total_turbine_cost + total_wiring_cost

toolbox.register("evaluate", evaluate)

In [ ]:
def clamp_reflect(_min, _max):
    def wrapper(func):
        def wrapped(*args, **kwargs):
            inds: tuple = func(*args, **kwargs)
            for ind in inds:
                for i in range(len(ind)):
                    if ind[i] > _max:
                        ind[i] = 2 * _max - ind[i]
                    elif ind[i] < _min:
                        ind[i] = 2 * _min - ind[i]
            return inds
        return wrapped
    return wrapper

In [ ]:

def mutate(ind):
    if random.random() < 0.8:
        ind, = tools.mut_gaussian(ind, 0, 0.05 * site.boundary_radius, 1.0 / len(ind))
    else:
        if (random.random() < 0.5 and len(ind) < MAX_TURBINES * 2) or len(ind) == MIN_TURBINES * 2:
            for _ in range(2):
                ind.append(toolbox.attr_coordinate())
        else:
            del ind[-2:]
    return ind,

toolbox.register("mutate", mutate)
toolbox.decorate("mutate", clamp_reflect(-boundary_length, boundary_length))


In [ ]:
toolbox.register("mate", tools.cx_two_point)
toolbox.decorate("mate", clamp_reflect(-boundary_length, boundary_length))

toolbox.register("select", tools.sel_tournament, contestants=4)

In [ ]:
NUM_GEN = 2000
POP_SIZE = 100
PC = 0.5
PM = 0.9

pop = toolbox.population(POP_SIZE)
pop, logbook = evolve(toolbox, pop, pc=PC, pm=PM, num_elitism=round(POP_SIZE * 0.1), num_gen=NUM_GEN)    

In [ ]:
pareto_front = logbook.select("pareto_front")[-1]

for ind in pareto_front:
    x, y = ind[::2], ind[1::2]
    s = boundary_length * 1.25
    plt.xlim(-s, s)
    plt.ylim(-s, s)
    plt.scatter(x, y)
    plt.show()

In [ ]:
pfs = logbook.select("pareto_front")
pfss = [pfs[0], pfs[-1]]

for pf in pfss:
    x=[]
    y=[]
    for ind in pf:
        x.append(ind.fitness.values[0])
        y.append(ind.fitness.values[1])
    plt.ylim(0, 3 * 10**7)
    plt.xlim(0, 3 * 10**8) 
    plt.scatter(x, y)
    plt.show()
    

In [ ]:
print(max_cost, max_output)